This is an introductory notebook showing how to work with the Google Earth Engine Python API. First, run the following cell to import and initialize the Earth Engine library.

In [1]:
import ee
ee.Authenticate()
ee.Initialize()
print(ee.Image("NASA/NASADEM_HGT/001").get("title").getInfo())

Enter verification code: 4/1AdQt8qhzlSr_48hpELTnoq2uW8VCTr3I5FGR1D00Vy_DjM7Hshl_WFOdrH4

Successfully saved authorization token.
NASADEM: NASA NASADEM Digital Elevation 30m


Import land cover, land surface temperature, and elevation datasets.

In [21]:
# import MODIS land cover collection
lc = ee.ImageCollection('MODIS/006/MCD12Q1')

# import MODIS land surface temperature collection
lst = ee.ImageCollection('MODIS/061/MOD11A1')

# import SRTM elevation data
elev = ee.Image('USGS/SRTMGL1_003')

In [22]:
# initial date (inclusive)
date1 = '2017-01-01'

# final date (exclusive)
date2 = '2020-01-01'

# select bands and filter by dates
lst = lst.select('LST_Day_1km', 'QC_Day').filterDate(date1, date2)

In [23]:
# define the urban location of interest as a point near Lyon, France
u_lon = 4.8148
u_lat = 45.7758
u_point = ee.Geometry.Point(u_lon, u_lat)

# define the rural location of interest as a point 30 km away from the city
r_lon = 5.175964
r_lat = 45.574064
r_point = ee.Geometry.Point(r_lon, r_lat)

Sample the land cover, temperature, and elevation at the urban point.

In [24]:
scale = 1000  # meters

# print elevation of Lyon, France
elev_u_point = elev.sample(u_point, scale).first().get('elevation').getInfo()
print('Ground elevation at Lyon, France:', elev_u_point, 'm')

# calculate & print the mean surface temperature at that point
lst_u_point_mean = lst.mean().sample(u_point, scale).first().get('LST_Day_1km').getInfo()
lst_u_point_mean = round(lst_u_point_mean*0.02 - 273.15, 2)
print('Land temperature at Lyon, France:', lst_u_point_mean, '°C')

# print land cover type at this point
lc_u_point = lc.first().sample(u_point, scale).first().get('LC_Type1').getInfo()
print('Land cover value at Lyon, France:', lc_u_point)

Ground elevation at Lyon, France: 196 m
Land temperature at Lyon, France: 23.26 °C
Land cover value at Lyon, France: 13


Sample the land cover, temperature, and elevation at the rural point.

In [25]:
scale = 1000  # meters

# print elevation of Lyon, France
elev_r_point = elev.sample(r_point, scale).first().get('elevation').getInfo()
print('Ground elevation at Lyon, France:', elev_r_point, 'm')

# calculate & print the mean surface temperature at that point
lst_r_point_mean = lst.mean().sample(r_point, scale).first().get('LST_Day_1km').getInfo()
lst_r_point_mean = round(lst_r_point_mean*0.02 - 273.15, 2)
print('Land temperature at Lyon, France:', lst_r_point_mean, '°C')

# print land cover type at this point
lc_r_point = lc.first().sample(r_point, scale).first().get('LC_Type1').getInfo()
print('Land cover value at Lyon, France:', lc_r_point)

Ground elevation at Lyon, France: 385 m
Land temperature at Lyon, France: 22.02 °C
Land cover value at Lyon, France: 12


In [10]:
# get data for the pixel intersecting the urban point
lst_u_point = lst.getRegion(u_point, scale).getInfo()

# get data for the pixel intersecting the rural point
lst_r_point = lst.getRegion(r_point, scale).getInfo()

# preview the information
lst_u_point[:5]

# QC_Day = 2 --> LST not calculated due to cloud effects

[['id', 'longitude', 'latitude', 'time', 'LST_Day_1km', 'QC_Day'],
 ['2017_01_01', 4.810478346460038, 45.77365530231022, 1483228800000, None, 2],
 ['2017_01_02', 4.810478346460038, 45.77365530231022, 1483315200000, None, 2],
 ['2017_01_03', 4.810478346460038, 45.77365530231022, 1483401600000, None, 2],
 ['2017_01_04',
  4.810478346460038,
  45.77365530231022,
  1483488000000,
  13808,
  17]]

In [11]:
# define a function to transform array into pandas dataframe
import pandas as pd

def ee_array_to_df(arr, list_of_bands):
    
    # transform client-side ee.Image.getRegion array to pandas.DataFrame
    df = pd.DataFrame(arr)
    
    # rearrange the header
    headers = df.iloc[0]
    df = pd.DataFrame(df.values[1:], columns=headers)
    
    # remove rows without data
    df = df[['longitude', 'latitude', 'time', *list_of_bands]].dropna()
    
    # convert data to numeric values
    for b in list_of_bands:
        df[b] = pd.to_numeric(df[b], errors='coerce')
        
    # convert time field into datetime
    df['datetime'] = pd.to_datetime(df['time'], unit='ms')
    
    # keep columns of interest
    df = df[['time', 'datetime', *list_of_bands]]
    
    return df

In [12]:
# define function to convert modis temp (kelvin scaled by 200) to celsius
def t_modis_to_celsius(t_modis):
    t_celsius = 0.02*t_modis - 273.15
    return t_celsius

In [13]:
# convert to dataframe
lst_df_urban = ee_array_to_df(lst_u_point,['LST_Day_1km'])
lst_df_rural = ee_array_to_df(lst_r_point,['LST_Day_1km'])

# get temp in celsius
lst_df_urban['LST_Day_1km'] = lst_df_urban['LST_Day_1km'].apply(t_modis_to_celsius)
lst_df_rural['LST_Day_1km'] = lst_df_rural['LST_Day_1km'].apply(t_modis_to_celsius)

lst_df_urban.head()

,time,datetime,LST_Day_1km
3,1483488000000,2017-01-04,3.01
4,1483574400000,2017-01-05,2.39
5,1483660800000,2017-01-06,0.89
6,1483747200000,2017-01-07,-0.11
15,1484524800000,2017-01-16,2.33


In [15]:
lst_df_rural.head()

,time,datetime,LST_Day_1km
6,1483747200000,2017-01-07,-2.55
10,1484092800000,2017-01-11,3.23
17,1484697600000,2017-01-18,-1.89
19,1484870400000,2017-01-20,0.07
20,1484956800000,2017-01-21,0.45
